# Catchment area analysis of supermarkets

## Libraries and settings

In [20]:
# Libraries
import os
import time
import glob
import json
import folium
import requests
import platform
import numpy as np
import pandas as pd
import geopandas as gdp
import matplotlib.pyplot as plt
from openrouteservice import client
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import supermarket data

In [21]:
# Read OpenRouteService API key
with open(file='ors_token.txt', mode='r') as file:
    api_key = file.read()
# print(api_key)

# Import supermarket data
df_orig = pd.read_csv('supermarkets_data_enriched.csv', 
                      sep=',',
                      encoding='utf-8')[['id',
                                         'lat',
                                         'lon',
                                         'brand',
                                         'bfs_number',
                                         'bfs_name',
                                         'addr:housenumber',
                                         'addr:postcode',
                                         'geometry']]

# Subset, only supermarkets with complete address
df = df_orig.dropna()
print(df.shape)
df.head()

(967, 9)


,id,lat,lon,brand,bfs_number,bfs_name,addr:housenumber,addr:postcode,geometry
1,280130028,47.155492,9.039666,ALDI,3315,Schänis,3,8718.0,POINT (9.0396663 47.1554921)
2,6122906632,47.158959,9.044477,Denner,3315,Schänis,14,8718.0,POINT (9.0444769 47.1589589)
4,36726161,47.226191,8.980329,Migros,3339,Uznach,25,8730.0,POINT (8.9803292 47.2261912)
6,7389830915,47.225056,8.970650,Denner,3339,Uznach,15,8730.0,POINT (8.9706499 47.2250563)
7,39947904,47.376732,8.542161,Coop,261,Zürich,1,8001.0,POINT (8.5421608 47.3767316)


## Create isochrone for a single supermarket

In [23]:
# Settings
municip = 'Adliswil'
transport = 'driving-car' # alternativle 'foot-walking'
traveltime = 20

# Client settings
ors = client.Client(key=api_key)

# Set up supermarket dictionary with a single supermarket
df_sub = df.loc[df['bfs_name'] == municip].iloc[0]
supermarkets = {df_sub['brand']: {'location': [df_sub['lon'], 
                                               df_sub['lat']]}}
print(supermarkets)

# Set up folium map
map = folium.Map(tiles='openstreetmap', 
                 location=([df_sub['lat'], 
                            df_sub['lon']]), 
                 zoom_start=11)

# Parameters for server-request
params_iso = {'profile': transport,
              'range': [traveltime*60],
              'attributes': ['total_pop']}

# Server request
for name, apt in supermarkets.items():
    
    # Add coords to request parameters
    params_iso['locations'] = [apt['location']]
    
    # Perform isochrone request
    apt['iso'] = ors.isochrones(**params_iso)
    
    # Add GeoJson to map
    folium.features.GeoJson(apt['iso']).add_to(map)
    
    # Save GeoJson as file
    # with open(f'{name}.json', 'w') as f:
        # f.write(json.dumps(apt['iso']))

    # Reverse coords due to weird folium lat/lon syntax
    folium.map.Marker(list(reversed(apt['location'])),
                      icon = folium.Icon(color='green',
                                         icon_color='#cc0000',
                                         icon='home',
                                         prefix="fa"),
                      popup = name
                      ).add_to(map)

# Plot map
map

{'Denner': {'location': [8.5218139, 47.3203453]}}


## Getting the number of residents in the isochrone area

In [ ]:
# Save the data as pandas data frame
data = pd.DataFrame(apt['iso']["features"])

# Create data frame from column 'properties'
df_pop = pd.DataFrame(data.loc[0, 'properties'])
val = df_pop['total_pop'][0]
print('Number of residents in isochrone area:', f'{val:,.0f}')

Number of residents in isochrone area: 163,756


## Estimating the purchasing power in the isochrone area

In [ ]:
# Simple assumtions:
# 500 CHF per resident and month
# Residents / employee ratio = 2:1
# 20 CHF per employee and working day

# Result
val = ((df_pop['total_pop'][0] * 500) + (df_pop['total_pop'][0] * 0.5 * 20 * 20)) / 10**6
print('Purchasing power per month:',
      f'{val:.1f}',
     'Mio. CHF')

Purchasing power per month: 114.6 Mio. CHF


## Automization of the analysis for multiple of supermarkets

In [ ]:
# Client settings
ors = client.Client(key=api_key)

# Parameters for server-request
params_iso = {'profile': 'driving-car',
              'range': [15*60],
              'attributes': ['total_pop']}

# Df to store the data
pop_out = []

try:
    for i in range(0, 10):
        
        print('Preparing isochrone for supermarkt', df[['id', 
                                                        'brand', 
                                                        'bfs_name', 
                                                        'addr:housenumber',
                                                        'addr:postcode']].iloc[i])
        
        supermarkets = {df['brand'].iloc[i]: {'location': [df['lon'].iloc[i],
                                                           df['lat'].iloc[i]]}}
        # Server request
        for name, apt in supermarkets.items():
            
            # Add coords to request parameters
            params_iso['locations'] = [apt['location']]
            
            # Perform isochrone request
            apt['iso'] = ors.isochrones(**params_iso)
            time.sleep(1)
            
            # Save the data as pandas data frame
            data = pd.DataFrame(apt['iso']["features"])
            
            # Create data frame from column 'properties'
            df_pop = pd.DataFrame(data.loc[0, 'properties'])
            
            # Write values
            pop_out.append([df['id'].iloc[i], 
                            df['brand'].iloc[i],
                            df['bfs_number'].iloc[i], 
                            df['bfs_name'].iloc[i], 
                            df['addr:housenumber'].iloc[i],
                            df['addr:postcode'].iloc[i],
                            df_pop['total_pop'][0],
                            ((df_pop['total_pop'][0] * 500) + (df_pop['total_pop'][0] * 0.5 * 20 * 20)) / 10**6])
            clear_output(wait=True)
            
except:
    pop_out.append([df['id'].iloc[i], 
                    df['brand'].iloc[i],
                    df['bfs_number'].iloc[i], 
                    df['bfs_name'].iloc[i], 
                    df['addr:housenumber'].iloc[i],
                    df['addr:postcode'].iloc[i],
                    np.NaN,
                    np.NaN])
    clear_output(wait=True)

# Print list
pop_out

[[280130028, 'ALDI', 3315, 'Schänis', '3', 8718.0, 76360.0, 53.452],
 [6122906632, 'Denner', 3315, 'Schänis', '14', 8718.0, 74635.0, 52.2445],
 [36726161, 'Migros', 3339, 'Uznach', '25', 8730.0, 0.0, 0.0],
 [7389830915, 'Denner', 3339, 'Uznach', '15', 8730.0, 60587.0, 42.4109],
 [39947904, 'Coop', 261, 'Zürich', '1', 8001.0, 493707.0, 345.5949],
 [48932835, 'Migros', 261, 'Zürich', '7', 8004.0, 494658.0, 346.2606],
 [75749133, 'ALDI', 261, 'Zürich', '81', 8038.0, 330520.0, 231.364],
 [79977755, 'Coop', 261, 'Zürich', '15', 8038.0, 323882.0, 226.7174],
 [83330862, 'Migros', 261, 'Zürich', '3', 8038.0, 322576.0, 225.8032],
 [119249170, 'Migros', 261, 'Zürich', '31-35', 8001.0, 484053.0, 338.8371]]

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.2.0-1019-azure
Datetime: 2024-03-12 20:26:11
Python Version: 3.10.13
-----------------------------------
